<a href="https://colab.research.google.com/github/dapsavoie/tz_water_well_prediction/blob/master/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries 

In [0]:
import pandas as pd 
import numpy as np

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Train Test Split

In [0]:
#Train dummies 
train_dummies = pd.read_csv(r'/content/drive/My Drive/tz_water_well_data/train_dummmies.csv')

#Test export to csv 
test_dummies = pd.read_csv(r'/content/drive/My Drive/tz_water_well_data/test_dummies.csv')

In [42]:
train_dummies.shape, test_dummies.shape

((59400, 315), (14850, 315))

# XGBOOST

In [0]:
labels = df_labels['status_group']

x = train_dummies

In [0]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [0]:
scaler = StandardScaler()

In [0]:
scaled_df = scaler.fit_transform(x)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_df, labels)

In [48]:
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
training_preds = clf.predict(X_train)
val_preds = clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

Training Accuracy: 74.64%
Validation accuracy: 74.97%


In [56]:
test_preds = clf.predict(test_dummies, validate_features=False)
val_accuracy = accuracy_score(y_test, test_preds)
val_accuracy

0.5437710437710438

In [0]:
# Grid Search CV 

from sklearn.model_selection import GridSearchCV

In [0]:
param_grid = {
    "learning_rate": [0.1],
    'max_depth': [6],
    'min_child_weight': [10],
    'subsample': [ 0.7],
    'n_estimators': [5, 30, 100, 250],
}

In [0]:
grid_clf = GridSearchCV(clf, param_grid, scoring='accuracy', cv=None, n_jobs=1)
grid_clf.fit(scaled_df, labels)

best_parameters = grid_clf.best_params_

print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

training_preds = grid_clf.predict(X_train)
val_preds = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("")
print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))